**Predict the sentiment of tweets using Stanford CoreNLP library**

---



In [0]:
import pandas as pd
import numpy as np
import nltk
import re

In [0]:
df=pd.read_csv('last.csv')

In [0]:
 pd.options.display.max_colwidth = 250

In [0]:
df.head(1)

,Unnamed: 0,Date,Time,hashtags,tagged_usernames,Text,tokens,data,pol_sub,polarity,subjectivity
0,0,2020-05-20,11:47:00,"['#bjp', '#rss']",['@any_morya:'],"['big', 'shame', 'bjp', 'rss', 'tried', 'divide', 'hindu', 'muslim', 'sunlight', 'heat', 'keeping', 'daily', 'ramzan', 'fasting']","['big', 'shame', 'bjp', 'rss', 'tried', 'divide', 'hindu', 'muslim', 'sunlight', 'heat', 'keeping', 'daily', 'ramzan', 'fasting']",its big shame to bjp rss who tried to divide hindu muslim in such sunlight and heat keeping their daily ramzan fasting,"Sentiment(polarity=0.0, subjectivity=0.19999999999999998)",0.0,0.2


In [0]:
pip install pycorenlp

  Created wheel for pycorenlp: filename=pycorenlp-0.3.0-cp36-none-any.whl size=2143 sha256=92890fe2198e689e0911e6aef4f6dd52fb94a60759c195ede82db51eaced48a3
  Stored in directory: /root/.cache/pip/wheels/fb/e9/2f/767a7b5f2e82d587a36143c04a21839b4b14bebfb89410d2d5
Successfully built pycorenlp


In [0]:
LOGDIR = '/tmp/log'
get_ipython().system_raw(
    """java -mx8g -cp "*" 
    edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9001 ->timeout 60000 &"""
    .format(LOGDIR)
)

In [0]:
! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip ngrok-stable-linux-amd64.zip

--2020-05-28 06:07:45--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 18.213.76.145, 52.201.33.182, 52.54.251.217, ...
Connecting to bin.equinox.io (bin.equinox.io)|18.213.76.145|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  6.59MB/s    in 2.0s    

2020-05-28 06:07:48 (6.59 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [0]:
get_ipython().system_raw('./ngrok http 9001 &')

In [0]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://b7b1618c6213.ngrok.io


In [0]:
from pycorenlp import StanfordCoreNLP

In [0]:
# Install stanza; note that the prefix "!" is not needed if you are running in a terminal
!pip install stanza

# Import stanza
import stanza

     |████████████████████████████████| 194kB 2.8MB/s 


In [0]:
# Download the Stanford CoreNLP Java library and unzip it to a ./corenlp folder
!echo "Downloading CoreNLP..."
!wget "http://nlp.stanford.edu/software/stanford-corenlp-4.0.0.zip" -O corenlp.zip
!unzip corenlp.zip
!mv ./stanford-corenlp-4.0.0 ./corenlp

# Set the CORENLP_HOME environment variable to point to the installation location
import os
os.environ["CORENLP_HOME"] = "./corenlp"

--2020-05-28 06:08:35--  http://nlp.stanford.edu/software/stanford-corenlp-4.0.0.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/software/stanford-corenlp-4.0.0.zip [following]
--2020-05-28 06:08:35--  https://nlp.stanford.edu/software/stanford-corenlp-4.0.0.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 504479415 (481M) [application/zip]
Saving to: ‘corenlp.zip’

corenlp.zip         100%[===================>] 481.11M  21.1MB/s    in 31s     

2020-05-28 06:09:06 (15.7 MB/s) - ‘corenlp.zip’ saved [504479415/504479415]

Archive:  corenlp.zip
   creating: stanford-corenlp-4.0.0/
  inflating: stanford-corenlp-4.0.0/jaxb-impl-2.4.0-b180830.0438.jar  
  inflating: stanford-corenlp-4.0.0/RESOURCE-LICENSES  
  inflat

In [0]:
# Import client module
from stanza.server import CoreNLPClient

In [0]:
# Construct a CoreNLPClient with some basic annotators, a memory allocation of 4GB, and port number 9001
client = CoreNLPClient(annotators=['tokenize','ssplit', 'pos', 'lemma', 'ner'], memory='4G', endpoint='http://localhost:9001')
print(client)

# Start the background server and wait for some time
# Note that in practice this is totally optional, as by default the server will be started when the first annotation is performed
client.start()
import time; time.sleep(10)

Starting server with command: java -Xmx4G -cp ./corenlp/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9001 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-0f5fd2eb8d384d2a.props -preload tokenize,ssplit,pos,lemma,ner


In [0]:
# Print background processes and look for java
!ps -o pid,cmd | grep java

    285 java -Xmx4G -cp ./corenlp/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9001 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-0f5fd2eb8d384d2a.props -preload tokenize,ssplit,pos,lemma,ner
    315 /bin/bash -c ps -o pid,cmd | grep java


In [0]:
nlp = StanfordCoreNLP('http://localhost:9001')

In [0]:
df.shape

(2699, 11)

In [0]:
 df['data'].replace('', np.nan, inplace=True)

In [0]:
df.dropna(subset=['data'], inplace=True)

In [0]:
df.shape

(2672, 11)

In [0]:
del df['Unnamed: 0']

In [0]:
df = df.reset_index(drop=True)

In [0]:
df.shape

(2672, 10)

In [0]:
df=df.assign(Stanford_score=np.nan,Stanford_sentiment=np.nan)

In [0]:
for i in range(2624,2672):
  res = nlp.annotate(df['data'][i],
                    properties={
                        'annotators': 'sentiment',
                        'outputFormat': 'json',
                        'timeout': 1000,
                    })
  for s in res["sentences"]:
      print(i)
      print("%d: '%s': %s %s" % (
          s["index"],
          " ".join([t["word"] for t in s["tokens"]]),
          s["sentimentValue"], s["sentiment"]))


2624
0: 'before august when i told people i was running for congress they thought it was a joke when they saw me leadbyexam': 1 Negative
2625
0: 'blm skirts management failures asks congress to accelerate program collapse action item tue': 1 Negative
2626
0: 'more evidence the whitehouse and congress aren currently interested in another round phase of stimulus': 2 Neutral
2627
0: 'if americans can serve selflessly throughout the coronavirus crisis and if the white house can continue to go to work every d': 1 Negative
2628
0: 'priyankabusscam auto rickshaws ambulances small capacity school vans amp vehicles with expired fitness certificates were': 1 Negative
2629
0: 'uppolice detain state congress chief ajaykumarlallu': 2 Neutral
2630
0: 'retweet if u agree that photographing during the compulsion of the poor has been an old hobby of the gandhi family congr': 1 Negative
2631
0: 'just in in the list of buses submitted by the congress to the up government have been found to be buses while

In [0]:
for i in range(2624,2672):
  res = nlp.annotate(df['data'][i],
                    properties={
                        'annotators': 'sentiment',
                        'outputFormat': 'json',
                        'timeout': 1000,
                    })
  for s in res["sentences"]:
          df['Stanford_score'][i]=s["sentimentValue"]
          df['Stanford_sentiment'][i]= s["sentiment"]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [0]:
df

,Date,Time,hashtags,tagged_usernames,Text,tokens,data,pol_sub,polarity,subjectivity,Stanford_score,Stanford_sentiment
0,2020-05-20,11:47:00,"['#bjp', '#rss']",['@any_morya:'],"['big', 'shame', 'bjp', 'rss', 'tried', 'divide', 'hindu', 'muslim', 'sunlight', 'heat', 'keeping', 'daily', 'ramzan', 'fasting']","['big', 'shame', 'bjp', 'rss', 'tried', 'divide', 'hindu', 'muslim', 'sunlight', 'heat', 'keeping', 'daily', 'ramzan', 'fasting']",its big shame to bjp rss who tried to divide hindu muslim in such sunlight and heat keeping their daily ramzan fasting,"Sentiment(polarity=0.0, subjectivity=0.19999999999999998)",0.000000,0.200000,1.0,Negative
1,2020-05-20,11:47:00,[],"['@milligazette:', '@muslimmirror']","['fir', 'news', 'website', '.', 'according', 'gujarat', 'police', 'day', 'shahpur', 'stone-pelting', 'video']","['fir', 'news', 'website', '.', 'according', 'gujarat', 'police', 'day', 'shahpur', 'stone-pelting', 'video']",fir against news website according to gujarat police a day after the shahpur stonepelting a video was,"Sentiment(polarity=0.0, subjectivity=0.0)",0.000000,0.000000,2.0,Neutral
2,2020-05-20,11:48:00,"['#bjp', '#gujrati', '#haridwar', '#covid__19.are']",[],"['bjp', 'govt', 'allows', '18', 'luxury', 'coaches', 'gujrati', 'pilgrims', 'haridwar', 'gujarat', 'amidst', 'covid__19.are', 'ppl']","['bjp', 'govt', 'allows', '18', 'luxury', 'coaches', 'gujrati', 'pilgrims', 'haridwar', 'gujarat', 'amidst', 'covid__19.are', 'ppl']",bjp govt allows luxury coaches for gujrati pilgrims from haridwar to gujarat amidst covid are the ppl of,"Sentiment(polarity=0.0, subjectivity=0.0)",0.000000,0.000000,2.0,Neutral
3,2020-05-20,11:46:00,"['#bjp', '#rss', '#rss_terrorists']",['@muslimmirror:'],"['armed', 'bjp', 'rss', 'activists', 'police', 'shahpur', 'ahmedabad', 'rss_terrorists']","['armed', 'bjp', 'rss', 'activists', 'police', 'shahpur', 'ahmedabad', 'rss_terrorists']",armed bjp rss activists with police in shahpur ahmedabad rss terrorists,"Sentiment(polarity=0.0, subjectivity=0.0)",0.000000,0.000000,2.0,Neutral
4,2020-05-20,11:46:00,"['#hindurashtra', '#india', '#hindus', '#modi.']",[],"['aspirants', 'hindurashtra', 'india', 'must', 'see', 'pathetic', 'conditions', 'poor', 'hindus', 'modi', '.', 'worst']","['aspirants', 'hindurashtra', 'india', 'must', 'see', 'pathetic', 'conditions', 'poor', 'hindus', 'modi', '.', 'worst']",aspirants of hindurashtra in india must see pathetic conditions of poor hindus under modi in worst,"Sentiment(polarity=-0.7999999999999999, subjectivity=0.8666666666666667)",-0.800000,0.866667,1.0,Negative
...,...,...,...,...,...,...,...,...,...,...,...,...
2667,2020-05-20,01:27:00,[],['@ronnielouise2:'],"['dem', 'congress', 'see', 'ca25', 'went', 'red', '22', 'years', 'rep', 'wi', 'winning', 'votes', 'ano']","['dem', 'congress', 'see', 'ca25', 'went', 'red', '22', 'years', 'rep', 'wi', 'winning', 'votes', 'ano']",any dem in congress who does not see that ca went red after years and the rep in wi winning who votes for ano,"Sentiment(polarity=0.25, subjectivity=0.375)",0.250000,0.375000,3.0,Positive
2668,2020-05-20,01:27:00,['#congress'],['@vernbuchanan'],"['corrupt', 'congress', 'hard', 'work', '.', 'vote', 'stand', 'alone', 'unconstitutionality']","['corrupt', 'congress', 'hard', 'work', '.', 'vote', 'stand', 'alone', 'unconstitutionality']",corrupt congress hard at work did you vote for this stand alone unconstitutionality too or do you,"Sentiment(polarity=-0.39583333333333337, subjectivity=0.7708333333333333)",-0.395833,0.770833,1.0,Negative
2669,2020-05-20,01:26:00,['#tx10'],"['@harrisdemocrats', '@priteshgandhimd']","['case', 'anyone', 'wondering', 'tx10', 'debate', 'going', '...', 'killing', '.']","['case', 'anyone', 'wondering', 'tx10', 'debate', 'going', '...', 'killing', '.']",in case anyone is wondering how the tx debate is going is killing it,"Sentiment(polarity=0.0, subjectivity=0.0)",0.000000,0.000000,2.0,Neutral
2670,2020-05-20,01:24:00,"['#congress', '#nationalguard']",['@maddow:'],"['.', 'c

**Save the final csv file to 'last1.csv'**

In [0]:
df.to_csv('last1.csv')